## Using pandas on the MovieLens dataset

To show pandas in a more "applied" sense, let's use it to answer some questions about the [MovieLens](http://www.grouplens.org/datasets/movielens/) dataset. Recall that we've already read our data into DataFrames and merged it.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline

In [ ]:
# pass in column names for each CSV
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv('data/ml-100k/u.user', sep='|', names=u_cols)

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('data/ml-100k/u.data', sep='\t', names=r_cols)

# the movies file contains columns indicating the movie's genres
# let's only load the first five columns of the file with usecols
m_cols = ['movie_id', 'title', 'release_date', 'video_release_date', 'imdb_url']
movies = pd.read_csv('data/ml-100k/u.item', sep='|', names=m_cols, usecols=range(5))

# create one merged DataFrame
movie_ratings = pd.merge(movies, ratings)
lens = pd.merge(movie_ratings, users)

**What are the 25 most rated movies?**

In [ ]:
most_rated = lens.groupby('title').size().sort_values(ascending=False)[:25]
most_rated

There's a lot going on in the code above, but it's very idomatic. We're splitting the DataFrame into groups by movie title and applying the `size` method to get the count of records in each group. Then we order our results in descending order and limit the output to the top 25 using Python's slicing syntax.

In SQL, this would be equivalent to:

    SELECT title, count(1)
    FROM lens
    GROUP BY title
    ORDER BY 2 DESC
    LIMIT 25;

Alternatively, pandas has a nifty `value_counts` method - yes, this is simpler - the goal above was to show a basic `groupby` example.

In [ ]:
lens.title.value_counts()[:25]

**Which movies are most highly rated?**

In [ ]:
movie_stats = lens.groupby('title').agg({'rating': [np.size, np.mean]})
movie_stats.head()

We can use the `agg` method to pass a dictionary specifying the columns to aggregate (as keys) and a list of functions we'd like to apply.

Let's sort the resulting DataFrame so that we can see which movies have the highest average score.

In [ ]:
# sort by rating average
movie_stats.sort_values(by=[('rating', 'mean')], ascending=False).head()

Because our columns are now a [MultiIndex](http://pandas.pydata.org/pandas-docs/stable/indexing.html#hierarchical-indexing-multiindex), we need to pass in a tuple specifying how to sort.

The above movies are rated so rarely that we can't count them as quality films. Let's only look at movies that have been rated at least 100 times.

In [ ]:
atleast_100 = movie_stats['rating']['size'] >= 100
movie_stats[atleast_100].sort_values(by=[('rating', 'mean')], ascending=False)[:15]

Those results look realistic.  Notice that we used boolean indexing to filter our `movie_stats` frame.

We broke this question down into many parts, so here's the Python needed to get the 15 movies with the highest average rating, requiring that they had at least 100 ratings:

```python
    movie_stats = lens.groupby('title').agg({'rating': [np.size, np.mean]})
    atleast_100 = movie_stats['rating'].size >= 100
    movie_stats[atleast_100].sort([('rating', 'mean')], ascending=False)[:15]
```

The SQL equivalent would be:

    SELECT title, COUNT(1) size, AVG(rating) mean
    FROM lens
    GROUP BY title
    HAVING COUNT(1) >= 100
    ORDER BY 3 DESC
    LIMIT 15;

**Limiting our population going forward**

Going forward, let's only look at the 50 most rated movies. Let's make a Series of movies that meet this threshold so we can use it for filtering later.

In [ ]:
most_50 = lens.groupby('movie_id').size().sort_values(ascending=False)[:50]

The SQL to match this would be:

    CREATE TABLE most_50 AS (
        SELECT movie_id, COUNT(1)
        FROM lens
        GROUP BY movie_id
        ORDER BY 2 DESC
        LIMIT 50
    );
    
This table would then allow us to use EXISTS, IN, or JOIN whenever we wanted to filter our results. Here's an example using EXISTS:

    SELECT *
    FROM lens
    WHERE EXISTS (SELECT 1 FROM most_50 WHERE lens.movie_id = most_50.movie_id);

**Which movies are most controversial amongst different ages?**

Let's look at how these movies are viewed across different age groups. First, let's look at how age is distributed amongst our users.

In [ ]:
users.age.head()

In [ ]:
users.age.hist(bins=30)
plt.title("Distribution of users' ages")
plt.ylabel('count of users')
plt.xlabel('age');

pandas' integration with [matplotlib](http://matplotlib.org/index.html) makes basic graphing of Series/DataFrames trivial.  In this case, just call `hist` on the column to produce a histogram.  We can also use [matplotlib.pyplot](http://matplotlib.org/users/pyplot_tutorial.html) to customize our graph a bit (always label your axes).

**Binning our users**

I don't think it'd be very useful to compare individual ages - let's bin our users into age groups using `pandas.cut`.

In [ ]:
labels = ['0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79']
lens['age_group'] = pd.cut(lens.age, range(0, 81, 10), right=False, labels=labels)
lens[['age', 'age_group']].drop_duplicates()[:10]

`pandas.cut` allows you to bin numeric data. In the above lines, we first created labels to name our bins, then split our users into eight bins of ten years (0-9, 10-19, 20-29, etc.). Our use of `right=False` told the function that we wanted the bins to be *exclusive* of the max age in the bin (e.g. a 30 year old user gets the 30s label).

Now we can now compare ratings across age groups.

In [ ]:
lens.groupby('age_group').agg({'rating': [np.size, np.mean]})

Young users seem a bit more critical than other age groups. Let's look at how the 50 most rated movies are viewed across each age group. We can use the `most_50` Series we created earlier for filtering.

In [ ]:
lens.set_index('movie_id', inplace=True)

In [ ]:
by_age = lens.ix[most_50.index].groupby(['title', 'age_group'])
by_age.rating.mean().head(15)

Notice that both the title and age group are indexes here, with the average rating value being a Series. This is going to produce a really long list of values.

Wouldn't it be nice to see the data as a table? Each title as a row, each age group as a column, and the average rating in each cell.

Behold! The magic of `unstack`!

In [ ]:
by_age.rating.mean().unstack(1).fillna(0)[10:20]

`unstack`, well, unstacks the specified level of a [MultiIndex](http://pandas.pydata.org/pandas-docs/stable/indexing.html#hierarchical-indexing-multiindex) (by default, `groupby` turns the grouped field into an index - since we grouped by two fields, it became a MultiIndex). We unstacked the second index (remember that Python uses 0-based indexes), and then filled in NULL values with 0.

If we had used:
```python
    by_age.rating.mean().unstack(0).fillna(0)
```
We would have had our age groups as rows and movie titles as columns.

**Which movies do men and women most disagree on?**

*Wes McKinney basically went through the exact same question in his book. It's a good, yet simple example of pivot_table. McKinney's book is the go-to text for pandas: [book here](http://www.amazon.com/gp/product/1449319793/ref=as_li_tl?ie=UTF8&camp=1789&creative=390957&creativeASIN=1449319793&linkCode=as2&tag=gjreda-20&linkId=MCGW4C4NOBRVV5OC).*

Think about how you'd have to do this in SQL for a second. You'd have to use a combination of IF/CASE statements with aggregate functions in order to pivot your dataset. Your query would look something like this:

    SELECT title, AVG(IF(sex = 'F', rating, NULL)), AVG(IF(sex = 'M', rating, NULL))
    FROM lens
    GROUP BY title;
    
Imagine how annoying it'd be if you had to do this on more than two columns.
    
DataFrame's have a *pivot_table* method that makes these kinds of operations much easier (and less verbose).

In [ ]:
lens.reset_index('movie_id', inplace=True)

In [ ]:
pivoted = lens.pivot_table(index=['movie_id', 'title'],
                           columns=['sex'],
                           values='rating',
                           fill_value=0)
pivoted.head()

In [ ]:
pivoted['diff'] = pivoted.M - pivoted.F
pivoted.head()

In [ ]:
pivoted.reset_index('movie_id', inplace=True)
pivoted.head()

In [ ]:
disagreements = pivoted[pivoted.movie_id.isin(most_50.index)]['diff']
disagreements.sort_values().plot(kind='barh', figsize=[9, 15])
plt.title('Male vs. Female Avg. Ratings\n(Difference > 0 = Favored by Men)')
plt.ylabel('Title')
plt.xlabel('Average Rating Difference')
plt.show()

Of course men like Terminator more than women. Independence Day though? Really?

### Additional Resources:

* [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/)
* [Introduction to pandas](http://nbviewer.ipython.org/urls/gist.github.com/fonnesbeck/5850375/raw/c18cfcd9580d382cb6d14e4708aab33a0916ff3e/1.+Introduction+to+Pandas.ipynb) by [Chris Fonnesbeck](https://twitter.com/fonnesbeck)
* [pandas videos from PyCon](http://pyvideo.org/search?models=videos.video&q=pandas)
* [pandas and Python top 10](http://manishamde.github.io/blog/2013/03/07/pandas-and-python-top-10/)
* [pandasql](http://blog.yhathq.com/posts/pandasql-sql-for-pandas-dataframes.html)
* [Practical pandas by Tom Augspurger (one of the pandas developers)](http://tomaugspurger.github.io/categories/pandas.html)
    * [Video](https://www.youtube.com/watch?v=otCriSKVV_8) from Tom's pandas tutorial at PyData Seattle 2015